<a href="https://colab.research.google.com/github/MarceloClaro/Computacao-Quantica-Geomaker/blob/main/Uma_Implementa%C3%A7%C3%A3o_de_Perceptron_Qu%C3%A2ntico_para_Classifica%C3%A7%C3%A3o_Bin%C3%A1ria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Uma Implementação de Perceptron Quântico para Classificação Binária**

---

**Introdução**

A computação quântica, uma abordagem emergente para processamento de informações, tem potencial para revolucionar áreas como otimização e aprendizado de máquina. Em nosso estudo, focamos na implementação de um perceptron quântico, uma variação quântica do perceptron clássico, para tarefas de classificação binária. Este ensaio descreve o processo de construção e avaliação de um perceptron quântico, apresentando os resultados obtidos.

---

**Metodologia**

Utilizamos a biblioteca Qiskit para criar e simular circuitos quânticos e a biblioteca scikit-learn para avaliar nosso modelo. Nosso processo envolveu:

1. **Configuração do simulador**: Configuramos o backend do Qiskit para simular circuitos quânticos.
   
2. **Normalização de dados**: Dado que as operações quânticas requerem ângulos específicos, normalizamos nossos dados para o intervalo \([0, \frac{\pi}{2}]\).
   
3. **Criação do perceptron quântico**: Criamos um circuito quântico que atua como um perceptron. Para cada característica de entrada, aplicamos rotações controladas pelos dados de entrada e pelos pesos associados a essa característica.
   
4. **Geração de dados**: Geramos dados fictícios, onde a classificação foi determinada pela média dos valores de entrada.
   
5. **Treinamento do perceptron**: Utilizando um loop de treinamento, iterativamente ajustamos os pesos do perceptron com base no erro entre a saída prevista e a saída real.
   
6. **Avaliação do modelo**: Avaliamos o modelo em um conjunto de teste usando métricas como matriz de confusão, curva ROC e acurácia.

---

**Resultados**

Apresentamos aqui os principais resultados obtidos:

1. **Matriz de Confusão**:
   
![Matriz de Confusão](/mnt/data/baixados (6).png)

Esta matriz mostra a relação entre as previsões do nosso modelo e os valores reais. Pode-se observar que o modelo conseguiu fazer previsões corretas em muitos casos, mas também houve vários erros.

2. **Curva ROC**:
   
![Curva ROC](/mnt/data/baixados (7).png)

A curva ROC indica a capacidade do modelo de distinguir entre as classes. A área sob a curva (AUC) é uma métrica importante, e neste caso, a AUC foi de 0.61.

3. **Acurácia**:
   
![Acurácia](/mnt/data/baixados (8).png)

A acurácia do modelo no conjunto de teste foi de 61%, o que indica um desempenho moderado.

4. **Circuito Quântico**:

O circuito quântico do perceptron é representado abaixo:

```
     ┌─────────────┐┌────────────┐        
q_0: ┤ Ry(0.27104) ├┤ Rz(446.64) ├──■─────
     └┬────────────┤├────────────┤  │     
q_1: ─┤ Ry(1.1211) ├┤ Rz(446.06) ├──■─────
     ┌┴────────────┤├────────────┤  │     
q_2: ┤ Ry(0.27104) ├┤ Rz(447.56) ├──■─────
     └─┬──────────┬┘├────────────┤  │     
q_3: ──┤ Ry(1.54) ├─┤ Rz(447.46) ├──■─────
       └──────────┘ └────────────┘┌─┴─┐┌─┐
q_4: ─────────────────────────────┤ X ├┤M├
                                  └───┘└╥┘
c: 1/═══════════════════════════════════╩═
                                        0
```

---

**Conclusão**

A implementação do perceptron quântico apresentou resultados interessantes, com uma acurácia de 61% em nosso conjunto de teste. Embora este resultado não seja excepcional, é importante lembrar que este é um exemplo simplificado, e otimizações adicionais podem ser aplicadas para melhorar o desempenho. A capacidade de construir e simular perceptrons quânticos abre a porta para explorar modelos mais complexos e otimizados na área da aprendizagem de máquina quântica.

In [ ]:
# Instalação das bibliotecas necessárias
!pip install qiskit
!pip install qiskit-aer
!pip install pylatexenc

# Importações
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit import *
from qiskit.visualization import *
from qiskit.circuit.library import MCPhaseGate
import qiskit.quantum_info as qi
import time



In [ ]:
# Importando bibliotecas necessárias
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, Aer, transpile, assemble
from qiskit.visualization import plot_histogram
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from qiskit import QuantumCircuit, transpile, execute, Aer
from sklearn.metrics import confusion_matrix, roc_curve, auc
import time



# Configurando o simulador
# Configura o backend do simulador do Qiskit para executar circuitos quânticos.
simulator = Aer.get_backend('aer_simulator')

# Definindo funções para normalizar e desnormalizar as fases
# Define funções para normalizar e desnormalizar valores,
# facilitando o mapeamento de valores para o intervalo de ângulos utilizável em operações quânticas.
def phase_normalize(value, min_value, max_value):
    return ((value - min_value) / (max_value - min_value)) * (np.pi / 2)

def phase_denormalize(normalized_value, min_value, max_value):
    return ((normalized_value * (max_value - min_value)) / (np.pi / 2)) + min_value

# Função para criar o circuito perceptron quântico
# Define uma função para criar um circuito quântico que representa um perceptron,
# usando rotações controladas pelos dados de entrada e pesos.
def create_perceptron_circuit(input_values, weights, n):
    reg_q = QuantumRegister(n + 1, 'q')  # Adicionado 1 para acomodar o qubit de destino extra
    reg_c = ClassicalRegister(1, 'c')
    circuit = QuantumCircuit(reg_q, reg_c)

    # Supondo que input_values e weights são normalizados para o intervalo [0, pi/2]
    for i in range(n):
        circuit.ry(2 * input_values[i], reg_q[i])  # Rotaciona os qubits de acordo com os valores de entrada
        circuit.rz(2 * weights[i], reg_q[i])       # Aplica uma rotação com base nos pesos

    # Operação de soma controlada (esta é uma simplificação, na prática, você pode precisar de uma operação de soma mais complexa)
    control_qubits = [reg_q[i] for i in range(n)]
    target_qubit = reg_q[n]  # O último qubit é usado como qubit de destino
    circuit.mcx(control_qubits, target_qubit)

    # Medição
    circuit.measure(target_qubit, reg_c)

    return circuit

# Gerando dados fictícios
def generate_fictitious_data(num_samples):
    data_x = np.random.randint(0, 256, size=(num_samples, 4))
    data_y = np.where(data_x.mean(axis=1) > 128, 1, 0)
    return data_x, data_y


# Normalizar os dados
train_set_x, train_set_y = generate_fictitious_data(1000)
test_set_x, test_set_y = generate_fictitious_data(200)
train_set_x_normalizado = phase_normalize(train_set_x, 0, 255)
test_set_x_normalizado = phase_normalize(test_set_x, 0, 255)


# Inicializar pesos
n = 4
pesos = np.random.uniform(0, np.pi/2, size=n)

# Parâmetros de treinamento
taxa_aprendizado = 0.005
num_epocas = 100

inicio = time.time()

# Loop de treinamento
for epoca in range(num_epocas):
    for i in range(len(train_set_x)):
        # Criar o circuito
        circuito = create_perceptron_circuit(train_set_x_normalizado[i], pesos, n)

        # Simular o circuito
        circuito_compilado = transpile(circuito, simulator)
        trabalho = simulator.run(circuito_compilado, shots=1000)
        resultado = trabalho.result()
        contagens = resultado.get_counts()

        # Calcular o erro
        previsto = contagens.get('1', 0) / 1000
        erro = previsto - train_set_y[i]

        # Atualizar pesos
        pesos -= taxa_aprendizado * erro

# Avaliação
previsoes = []
for i in range(len(test_set_x)):
    circuito = create_perceptron_circuit(test_set_x_normalizado[i], pesos, n)
    circuito_compilado = transpile(circuito, simulator)
    trabalho = simulator.run(circuito_compilado, shots=1000)
    resultado = trabalho.result()
    contagens = resultado.get_counts()
    previsto = 1 if contagens.get('1', 0) / 1000 > 0.5 else 0  # Limiar de 0.5
    previsoes.append(previsto)

fim = time.time()
tempo_decorrido = fim - inicio

# Matriz de Confusão
confusao = confusion_matrix(test_set_y, previsoes)
sns.heatmap(confusao, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Previsto')
plt.ylabel('Verdadeiro')
plt.title('Matriz de Confusão')
plt.show()

# Curva ROC
fpr, tpr, _ = roc_curve(test_set_y, previsoes)
roc_auc = auc(fpr, tpr)
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='Curva ROC (área = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Taxa de Falso Positivo')
plt.ylabel('Taxa de Verdadeiro Positivo')
plt.title('Curva ROC')
plt.legend(loc="lower right")
plt.show()

# Acurácia
acuracia = np.mean(np.array(previsoes) == test_set_y)
plt.figure(figsize=(10, 5))
plt.bar(['Acurácia do Teste'], [acuracia])
plt.ylim(0, 1)
plt.ylabel('Acurácia')
plt.title('Acurácia do Modelo')
plt.show()

print(f"Tempo de Processamento: {tempo_decorrido:.2f} segundos")
print(f'Acurácia do Teste: {acuracia:.2f}')
circuito.draw(output='text')


**Ensaio Acadêmico sobre a Classificação Quântica do Conjunto de Dados Iris**

---

**Introdução**

O conjunto de dados Iris, um dos conjuntos de dados mais bem conhecidos na literatura de aprendizado de máquina, contém medições de três espécies de flores. Este ensaio aborda um método inovador de classificação quântica para distinguir entre a espécie 'Iris-setosa' e as outras espécies no conjunto de dados. Utilizando a biblioteca Qiskit para simulação de circuitos quânticos e a biblioteca Scikit-learn para métricas de avaliação, exploramos a eficácia desse método.

---

**Metodologia**

1. **Preparação dos Dados**:
   - O conjunto de dados foi carregado com medições para comprimento e largura da sépala e da pétala, e o nome da espécie.
   - Os dados foram separados em características e rótulos. As características foram armazenadas na matriz \( X \) e os rótulos na matriz \( y \).

2. **Inicialização de Parâmetros**:
   - Pesos foram inicializados aleatoriamente no intervalo \([0, \frac{\pi}{2}]\) com dimensões correspondentes ao número de características.
   - Taxa de aprendizado e número de épocas foram definidos para o treinamento.

3. **Treinamento**:
   - Para cada época e cada amostra no conjunto de dados:
     - Um circuito quântico foi criado.
     - O circuito foi simulado e a saída foi interpretada como uma previsão.
     - O erro entre a previsão e o rótulo verdadeiro foi calculado.
     - Os pesos foram atualizados usando uma regra de atualização simples baseada no erro.

4. **Avaliação**:
   - A Curva ROC (Receiver Operating Characteristic) foi calculada e exibida.
   - Uma matriz de confusão foi gerada e exibida.
   - A perda de teste e a acurácia foram calculadas e exibidas.

---

**Resultados**

Os gráficos da Curva ROC, matriz de confusão e erros de teste, bem como a acurácia, foram gerados:

1. **Matriz de Confusão**:
   - A matriz de confusão foi visualizada usando a biblioteca Seaborn, fornecendo uma representação visual do desempenho do modelo.
   
   ![Matriz de Confusão](/mnt/data/baixados (12).png)

2. **Curva ROC**:
   - A Curva ROC ilustra a capacidade do modelo de discriminar entre classes positivas e negativas.
   
   ![Curva ROC](/mnt/data/baixados (11).png)

3. **Erro de Teste e Acurácia**:
   - A evolução do erro de teste ao longo das amostras e a acurácia geral do modelo foram exibidos.
   
   ![Erro e Acurácia](/mnt/data/baixados (9).png)

Valores finais:
- Perda de Teste: -0.01064935064935065
- Acurácia de Teste: 0.9176623376623376

---

**Discussão**

A abordagem de classificação quântica mostrou-se promissora, alcançando uma acurácia de aproximadamente 91.77%. O circuito quântico, que consiste em portas de rotação em torno do eixo Y, foi simulado usando o backend 'qasm_simulator' da biblioteca Qiskit. A Curva ROC indica que o modelo tem uma capacidade razoavelmente boa de discriminar entre as classes.

No entanto, vale ressaltar algumas limitações e considerações:
- O conjunto de dados Iris é relativamente simples e bem separado, portanto, os resultados podem não ser diretamente transferíveis para conjuntos de dados mais complexos.
- A simulação foi realizada em um ambiente clássico. A implementação em um computador quântico real pode apresentar desafios adicionais, como erros de qubit e limitações de coerência.

---

**Conclusão**

O ensaio explorou a aplicação da classificação quântica ao conjunto de dados Iris. Os resultados sugerem que a computação quântica tem potencial em tarefas de aprendizado de máquina, especialmente em conjuntos de dados que são intrinsecamente quânticos ou que se beneficiariam da paralelização quântica. Mais pesquisas e experimentos são necessários para validar e expandir essas descobertas em conjuntos de dados e problemas mais complexos.

In [ ]:

import numpy as np
from qiskit import QuantumCircuit, transpile, execute, Aer
from sklearn.metrics import roc_curve, auc, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Carregue o conjunto de dados Iris (usando os dados fornecidos)
# Certifique-se de ter carregado corretamente todos os exemplos de dados do conjunto Iris.
# Adicione mais exemplos de dados, se necessário, para treinamento e teste.
iris_data = [[5.1, 3.5, 1.4, 0.2, 'Iris-setosa'],
    [6.5, 3.0, 5.2, 2.0, 'Iris-versicolor'],
    [4.9, 3.0, 1.4, 0.2, 'Iris-setosa'],
    [4.7, 3.2, 1.3, 0.2, 'Iris-setosa'],
    [4.6, 3.1, 1.5, 0.2, 'Iris-setosa'],
    [4.9, 3.0, 1.4, 0.2, 'Iris-setosa'],
    [4.7, 3.2, 1.3, 0.2, 'Iris-setosa'],
    [4.6, 3.1, 1.5, 0.2, 'Iris-setosa'],
    [5.0, 3.6, 1.4, 0.2, 'Iris-setosa'],
    [5.4, 3.9, 1.7, 0.4, 'Iris-setosa'],
    [4.6, 3.4, 1.4, 0.3, 'Iris-setosa'],
    [5.0, 3.4, 1.5, 0.2, 'Iris-setosa'],
    [4.4, 2.9, 1.4, 0.2, 'Iris-setosa'],
    [4.9, 3.1, 1.5, 0.1, 'Iris-setosa'],
    [5.4, 3.7, 1.5, 0.2, 'Iris-setosa'],
    [4.8, 3.4, 1.6, 0.2, 'Iris-setosa'],
    [4.8, 3.0, 1.4, 0.1, 'Iris-setosa'],
    [4.3, 3.0, 1.1, 0.1, 'Iris-setosa'],
    [5.8, 4.0, 1.2, 0.2, 'Iris-setosa'],
    [5.7, 4.4, 1.5, 0.4, 'Iris-setosa'],
    [5.4, 3.9, 1.3, 0.4, 'Iris-setosa'],
    [5.1, 3.5, 1.4, 0.3, 'Iris-setosa'],
    [5.7, 3.8, 1.7, 0.3, 'Iris-setosa'],
    [5.1, 3.8, 1.5, 0.3, 'Iris-setosa'],
    [5.4, 3.4, 1.7, 0.2, 'Iris-setosa'],
    [5.1, 3.7, 1.5, 0.4, 'Iris-setosa'],
    [4.6, 3.6, 1.0, 0.2, 'Iris-setosa'],
    [5.1, 3.3, 1.7, 0.5, 'Iris-setosa'],
    [4.8, 3.4, 1.9, 0.2, 'Iris-setosa'],
    [5.0, 3.0, 1.6, 0.2, 'Iris-setosa'],
    [5.0, 3.4, 1.6, 0.4, 'Iris-setosa'],
    [5.2, 3.5, 1.5, 0.2, 'Iris-setosa'],
    [5.2, 3.4, 1.4, 0.2, 'Iris-setosa'],
    [4.7, 3.2, 1.6, 0.2, 'Iris-setosa'],
    [4.8, 3.1, 1.6, 0.2, 'Iris-setosa'],
    [5.4, 3.4, 1.5, 0.4, 'Iris-setosa'],
    [5.2, 4.1, 1.5, 0.1, 'Iris-setosa'],
    [5.5, 4.2, 1.4, 0.2, 'Iris-setosa'],
    [4.9, 3.1, 1.5, 0.1, 'Iris-setosa'],
    [5.0, 3.2, 1.2, 0.2, 'Iris-setosa'],
    [5.5, 3.5, 1.3, 0.2, 'Iris-setosa'],
    [4.9, 3.1, 1.5, 0.1, 'Iris-setosa'],
    [4.4, 3.0, 1.3, 0.2, 'Iris-setosa'],
    [5.1, 3.4, 1.5, 0.2, 'Iris-setosa'],
    [5.0, 3.5, 1.3, 0.3, 'Iris-setosa'],
    [4.5, 2.3, 1.3, 0.3, 'Iris-setosa'],
    [4.4, 3.2, 1.3, 0.2, 'Iris-setosa'],
    [5.0, 3.5, 1.6, 0.6, 'Iris-setosa'],
    [5.1, 3.8, 1.9, 0.4, 'Iris-setosa'],
    [4.8, 3.0, 1.4, 0.3, 'Iris-setosa'],
    [5.1, 3.8, 1.6, 0.2, 'Iris-setosa'],
    [4.6, 3.2, 1.4, 0.2, 'Iris-setosa'],
    [5.3, 3.7, 1.5, 0.2, 'Iris-setosa'],
    [5.0, 3.3, 1.4, 0.2, 'Iris-setosa'],
    [7.0, 3.2, 4.7, 1.4, 'Iris-versicolor'],
    [6.4, 3.2, 4.5, 1.5, 'Iris-versicolor'],
    [6.9, 3.1, 4.9, 1.5, 'Iris-versicolor'],
    [5.5, 2.3, 4.0, 1.3, 'Iris-versicolor'],
    [6.5, 2.8, 4.6, 1.5, 'Iris-versicolor'],
    [5.7, 2.8, 4.5, 1.3, 'Iris-versicolor'],
    [6.3, 3.3, 4.7, 1.6, 'Iris-versicolor'],
    [4.9, 2.4, 3.3, 1.0, 'Iris-versicolor'],
    [6.6, 2.9, 4.6, 1.3, 'Iris-versicolor'],
    [5.2, 2.7, 3.9, 1.4, 'Iris-versicolor'],
    [5.0, 2.0, 3.5, 1.0, 'Iris-versicolor'],
    [5.9, 3.0, 4.2, 1.5, 'Iris-versicolor'],
    [6.0, 2.2, 4.0, 1.0, 'Iris-versicolor'],
    [6.1, 2.9, 4.7, 1.4, 'Iris-versicolor'],
    [5.6, 2.9, 3.6, 1.3, 'Iris-versicolor'],
    [6.7, 3.1, 4.4, 1.4, 'Iris-versicolor'],
    [5.6, 3.0, 4.5, 1.5, 'Iris-versicolor'],
    [5.8, 2.7, 4.1, 1.0, 'Iris-versicolor'],
    [6.2, 2.2, 4.5, 1.5, 'Iris-versicolor'],
    [5.6, 2.5, 3.9, 1.1, 'Iris-versicolor'],
    [5.9, 3.2, 4.8, 1.8, 'Iris-versicolor'],
    [6.1, 2.8, 4.0, 1.3, 'Iris-versicolor'],
    [6.3, 2.5, 4.9, 1.5, 'Iris-versicolor'],
    [6.1, 2.8, 4.7, 1.2, 'Iris-versicolor'],
    [6.4, 2.9, 4.3, 1.3, 'Iris-versicolor'],
    [6.6, 3.0, 4.4, 1.4, 'Iris-versicolor'],
    [6.8, 2.8, 4.8, 1.4, 'Iris-versicolor'],
    [6.7, 3.0, 5.0, 1.7, 'Iris-versicolor'],
    [6.0, 2.9, 4.5, 1.5, 'Iris-versicolor'],
    [5.7, 2.6, 3.5, 1.0, 'Iris-versicolor'],
    [5.5, 2.4, 3.8, 1.1, 'Iris-versicolor'],
    [5.5, 2.4, 3.7, 1.0, 'Iris-versicolor'],
    [5.8, 2.7, 3.9, 1.2, 'Iris-versicolor'],
    [6.0, 2.7, 5.1, 1.6, 'Iris-versicolor'],
    [5.4, 3.0, 4.5, 1.5, 'Iris-versicolor'],
    [6.0, 3.4, 4.5, 1.6, 'Iris-versicolor'],
    [6.7, 3.1, 4.7, 1.5, 'Iris-versicolor'],
    [6.3, 2.3, 4.4, 1.3, 'Iris-versicolor'],
    [5.6, 3.0, 4.1, 1.3, 'Iris-versicolor'],
    [5.5, 2.5, 4.0, 1.3, 'Iris-versicolor'],
    [5.5, 2.6, 4.4, 1.2, 'Iris-versicolor'],
    [6.1, 3.0, 4.6, 1.4, 'Iris-versicolor'],
    [5.8, 2.6, 4.0, 1.2, 'Iris-versicolor'],
    [5.0, 2.3, 3.3, 1.0, 'Iris-versicolor'],
    [5.6, 2.7, 4.2, 1.3, 'Iris-versicolor'],
    [5.7, 3.0, 4.2, 1.2, 'Iris-versicolor'],
    [5.7, 2.9, 4.2, 1.3, 'Iris-versicolor'],
    [6.2, 2.9, 4.3, 1.3, 'Iris-versicolor'],
    [5.1, 2.5, 3.0, 1.1, 'Iris-versicolor'],
    [5.7, 2.8, 4.1, 1.3, 'Iris-versicolor'],
    [6.3, 3.3, 6.0, 2.5, 'Iris-virginica'],
    [5.8, 2.7, 5.1, 1.9, 'Iris-virginica'],
    [7.1, 3.0, 5.9, 2.1, 'Iris-virginica'],
    [6.3, 2.9, 5.6, 1.8, 'Iris-virginica'],
    [6.5, 3.0, 5.8, 2.2, 'Iris-virginica'],
    [7.6, 3.0, 6.6, 2.1, 'Iris-virginica'],
    [4.9, 2.5, 4.5, 1.7, 'Iris-virginica'],
    [7.3, 2.9, 6.3, 1.8, 'Iris-virginica'],
    [6.7, 2.5, 5.8, 1.8, 'Iris-virginica'],
    [7.2, 3.6, 6.1, 2.5, 'Iris-virginica'],
    [6.5, 3.2, 5.1, 2.0, 'Iris-virginica'],
    [6.4, 2.7, 5.3, 1.9, 'Iris-virginica'],
    [6.8, 3.0, 5.5, 2.1, 'Iris-virginica'],
    [5.7, 2.5, 5.0, 2.0, 'Iris-virginica'],
    [5.8, 2.8, 5.1, 2.4, 'Iris-virginica'],
    [6.4, 3.2, 5.3, 2.3, 'Iris-virginica'],
    [6.5, 3.0, 5.5, 1.8, 'Iris-virginica'],
    [7.7, 3.8, 6.7, 2.2, 'Iris-virginica'],
    [7.7, 2.6, 6.9, 2.3, 'Iris-virginica'],
    [6.0, 2.2, 5.0, 1.5, 'Iris-virginica'],
    [6.9, 3.2, 5.7, 2.3, 'Iris-virginica'],
    [5.6, 2.8, 4.9, 2.0, 'Iris-virginica'],
    [7.7, 2.8, 6.7, 2.0, 'Iris-virginica'],
    [6.3, 2.7, 4.9, 1.8, 'Iris-virginica'],
    [6.7, 3.3, 5.7, 2.1, 'Iris-virginica'],
    [7.2, 3.2, 6.0, 1.8, 'Iris-virginica'],
    [6.2, 2.8, 4.8, 1.8, 'Iris-virginica'],
    [6.1, 3.0, 4.9, 1.8, 'Iris-virginica'],
    [6.4, 2.8, 5.6, 2.1, 'Iris-virginica'],
    [7.2, 3.0, 5.8, 1.6, 'Iris-virginica'],
    [7.4, 2.8, 6.1, 1.9, 'Iris-virginica'],
    [7.9, 3.8, 6.4, 2.0, 'Iris-virginica'],
    [6.4, 2.8, 5.6, 2.2, 'Iris-virginica'],
    [6.3, 2.8, 5.1, 1.5, 'Iris-virginica'],
    [6.1, 2.6, 5.6, 1.4, 'Iris-virginica'],
    [7.7, 3.0, 6.1, 2.3, 'Iris-virginica'],
    [6.3, 3.4, 5.6, 2.4, 'Iris-virginica'],
    [6.4, 3.1, 5.5, 1.8, 'Iris-virginica'],
    [6.0, 3.0, 4.8, 1.8, 'Iris-virginica'],
    [6.9, 3.1, 5.4, 2.1, 'Iris-virginica'],
    [6.7, 3.1, 5.6, 2.4, 'Iris-virginica'],
    [6.9, 3.1, 5.1, 2.3, 'Iris-virginica'],
    [5.8, 2.7, 5.1, 1.9, 'Iris-virginica'],
    [6.8, 3.2, 5.9, 2.3, 'Iris-virginica'],
    [6.7, 3.3, 5.7, 2.5, 'Iris-virginica'],
    [6.7, 3.0, 5.2, 2.3, 'Iris-virginica'],
    [6.3, 2.5, 5.0, 1.9, 'Iris-virginica'],
    [6.5, 3.0, 5.2, 2.0, 'Iris-virginica'],
    [6.2, 3.4, 5.4, 2.3, 'Iris-virginica'],
    [5.9, 3.0, 5.1, 1.8, 'Iris-virginica'],
]


# Separando características e rótulos
X = np.array([data[:-1] for data in iris_data])
y = np.array([data[-1] for data in iris_data])

# Inicializando os pesos (certifique-se de ter pesos suficientes para as características)
n = X.shape[1]  # Número de características
weights = np.random.uniform(0, np.pi/2, size=n)

# Parâmetros de treinamento
learning_rate = 0.005
num_epochs = 100
inicio = time.time()
# Simulador
simulator = Aer.get_backend('qasm_simulator')

# Inicializando as listas
predictions = []
true_labels = []
errors = []


for epoch in range(num_epochs):
    for i in range(len(X)):
        # Criando o circuito
        circuit = QuantumCircuit(n, 1)
        for j in range(n):
            circuit.ry(weights[j], j)
        circuit.measure(0, 0)

        # Simulando o circuito
        transpiled_circuit = transpile(circuit, simulator)
        job = execute(transpiled_circuit, simulator, shots=1000)
        counts = job.result().get_counts()

        # Calculando a saída
        output = max(counts, key=counts.get)
        predicted = 1 if output == '1' else 0  # Converter para tipo numérico
        true = 1 if y[i] == 'Iris-setosa' else 0  # Converter para tipo numérico

        predictions.append(predicted)
        true_labels.append(true)

        # Calculando o erro
        error = predicted - true
        errors.append(error)

        # Atualizando os pesos
        weights -= learning_rate * error * X[i]


# Avaliação
# Cálculo da curva ROC
fpr, tpr, _ = roc_curve(true_labels, predictions)
roc_auc = auc(fpr, tpr)

# Cálculo da matriz de confusão
confusion = confusion_matrix(true_labels, predictions)
fim = time.time()
tempo_decorrido = fim - inicio
# Exibindo a matriz de confusão usando Seaborn
sns.heatmap(confusion, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

# Imprimindo a curva ROC
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.show()

# Calculando a acurácia
accuracy = 1 - np.mean(np.abs(errors))

# Criando uma nova figura
plt.figure(figsize=(10, 5))

# Plotando a perda de teste
plt.subplot(1, 2, 1)
plt.plot(errors)  # Usando 'errors' em vez de 'test_errors'
plt.title('Test Errors')
plt.xlabel('Sample Index')
plt.ylabel('Error')

# Plotando a acurácia
plt.subplot(1, 2, 2)
plt.bar(['Test Accuracy'], [accuracy])
plt.ylim(0, 1)  # Definindo o limite do eixo y para que vá de 0 a 1
plt.ylabel('Accuracy')

# Mostrando o gráfico
plt.show()

# Imprimindo a perda de teste e a acurácia
print(f'Test Loss: {np.mean(errors)}')  # Calculando a perda média
print(f"Tempo de Processamento: {tempo_decorrido:.2f} segundos")
print(f'Test Accuracy: {accuracy}')
circuit.draw()


**Plano de Aula: Introdução à Computação Quântica através da Classificação de Dados**

**Objetivo Geral**:
Apresentar aos alunos os conceitos básicos da computação quântica, demonstrando sua aplicação prática através da classificação do conjunto de dados Iris.

**Objetivos Específicos**:
1. Introduzir os conceitos fundamentais da computação quântica.
2. Familiarizar os alunos com o conjunto de dados Iris.
3. Demonstração prática da aplicação de um modelo quântico em um problema de classificação.

**Materiais Necessários**:
- Computadores com acesso à internet.
- Softwares de programação Python e Qiskit instalados.
- Projetor para demonstrações.

**Duração**: 2 aulas de 50 minutos cada.

---

**Aula 1: Introdução à Computação Quântica e Conjunto de Dados Iris**

1. **Abertura (10 minutos)**:
   - Discussão: "O que vocês sabem sobre computação quântica?"
   - Breve visão geral do que será abordado na aula.

2. **Introdução à Computação Quântica (15 minutos)**:
   - Explicação sobre o que é um qubit.
   - Discussão sobre superposição e entrelaçamento.
   - Demonstração básica de um circuito quântico usando Qiskit.

3. **Conjunto de Dados Iris (20 minutos)**:
   - Discussão sobre o que é classificação.
   - Apresentação do conjunto de dados Iris: características e classes.
   - Visualização dos dados: gráficos mostrando a distribuição das características.

4. **Encerramento (5 minutos)**:
   - Resumo do que foi aprendido.
   - Introdução ao que será abordado na próxima aula.

---

**Aula 2: Modelagem Quântica e Classificação de Dados**

1. **Abertura (10 minutos)**:
   - Revisão rápida dos conceitos da aula anterior.
   - Discussão: "Como vocês acham que podemos usar a computação quântica para classificar flores?"

2. **Modelo de Aprendizado Quântico (15 minutos)**:
   - Explicação sobre como os qubits podem ser usados para representar dados.
   - Discussão sobre como construir um circuito quântico para classificação.
   - Demonstração de um circuito quântico simples usando Qiskit.

3. **Treinamento e Avaliação (20 minutos)**:
   - Discussão sobre como os modelos são treinados: atualização de pesos.
   - Demonstração prática do código: treinando e avaliando o modelo quântico no conjunto de dados Iris.
   - Interpretação dos resultados: matriz de confusão, Curva ROC.

4. **Encerramento (5 minutos)**:
   - Discussão sobre as vantagens e limitações da computação quântica.
   - Encorajamento para os alunos explorarem mais sobre o tema e pensarem em outras aplicações.

---

**Avaliação**:
- Ao final das duas aulas, os alunos serão incentivados a refletir e discutir sobre as possíveis aplicações da computação quântica.
- Será proposto um exercício prático onde os alunos deverão modificar o código para classificar uma nova classe do conjunto de dados Iris, testando suas habilidades e compreensão do conteúdo apresentado.

**Dicas**:
- A abordagem deve ser interativa, estimulando a participação ativa dos alunos.
- Os conceitos quânticos podem ser complexos, então é importante garantir que os alunos estejam acompanhando e entendendo.
- Usar analogias e exemplos do mundo real pode ajudar a tornar o conteúdo mais acessível.